# Identification Accuracy

The **equation to calculate identification accuracy** in a GAN model is as follows:

**`Identification Accuracy = (Number of Correctly Identified Samples / Total Number of Samples) x 100%`**

In other words, you divide the number of samples that were correctly identified by the ***classifier or discriminator*** by the total number of samples in the dataset (both real and generated) and then multiply by 100% to get the accuracy percentage.

Here's an **example** of how to calculate identification accuracy using this formula:

Suppose you have a dataset of 1000 samples, with 500 real samples and 500 generated samples from your GAN model. You use a classifier or discriminator model to classify the samples and find that it correctly identifies 900 samples out of the 1000. Then the identification accuracy is:

`Identification Accuracy = (900 / 1000) x 100% = 90%`

Therefore, the *identification accuracy of your GAN model is 90%.*

In [1]:
import string
from dataset import VoiceDataset, FaceDataset
from network import VoiceEmbedNet, Generator, FaceEmbedNet, Classifier,get_network
from utils import get_collate_fn

DATASET_PARAMETERS = {
    # meta data provided by voxceleb1 dataset
    'meta_file': 'voxCeleb-Dataset\\vox1_meta.csv',

    # voice dataset
    'voice_dir': 'voxCeleb-Dataset\\fbank\\fbank',
    'voice_ext': 'npy',

    # face dataset
    'face_dir': 'voxCeleb-Dataset\\VGG_ALL_FRONTAL\\VGG_ALL_FRONTAL',
    'face_ext': '.jpg',

    # train data includes the identities
    # whose names start with the characters of 'FGH...XYZ'
    'split': string.ascii_uppercase[5:],

    # dataloader
    'voice_dataset': VoiceDataset,
    'face_dataset': FaceDataset,
    'batch_size': 128,
    'nframe_range': [300, 800],
    'workers_num': 1,
    'collate_fn': get_collate_fn,

    # test data
    'test_data': 'data\\example_data'
}


NETWORKS_PARAMETERS = {
    # VOICE EMBEDDING NETWORK (e)
    'e': {
        'network': VoiceEmbedNet,
        'input_channel': 64,
        'channels': [256, 384, 576, 864],
        'output_channel': 64,  # the embedding dimension
        'model_path': 'pretrained_models/voice_embedding.pth',
    },
    # GENERATOR (g)
    'g': {
        'network': Generator,
        'input_channel': 64,
        # channels for deconvolutional layers
        'channels': [1024, 512, 256, 128, 64],
        'output_channel': 3,  # images with RGB channels
        'model_path': 'models/generator.pth',
    },
    # FACE EMBEDDING NETWORK (f)
    'f': {
        'network': FaceEmbedNet,
        'input_channel': 3,
        'channels': [32, 64, 128, 256, 512],
        'output_channel': 64,
        'model_path': 'models/face_embedding.pth',
    },
    # DISCRIMINATOR (d)
    'd': {
        'network': Classifier,  # Discrminator is a special Classifier with 1 subject
        'input_channel': 64,
        'channels': [],
        'output_channel': 1,
        'model_path': 'models/discriminator.pth',
    },
    # CLASSIFIER (c)
    'c': {
        'network': Classifier,
        'input_channel': 64,
        'channels': [],
        'output_channel': -1,  # This parameter is depended on the dataset we used
        'model_path': 'models/classifier.pth',
    },
    # OPTIMIZER PARAMETERS
    'lr': 0.0002,
    'beta1': 0.5,
    'beta2': 0.999,

    # MODE, use GPU or not
    'GPU': True


}


c:\Users\Ahmed\.conda\envs\tensorflow-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
f_net, _ = get_network('f', NETWORKS_PARAMETERS, train=False) #face embed
d_net, _ = get_network('d', NETWORKS_PARAMETERS, train=False) #discriminator

In [3]:
def imread(filename):
    """
    Loads an image file into a (height, width, 3) uint8 ndarray.
    """
    image = Image.open(filename)
    img = image.resize((64, 64), Image.ANTIALIAS)
    return np.asarray(img, dtype=np.uint8)[..., :3]

In [19]:
from PIL import Image
import torch
from torchvision import transforms
import numpy as np

image = imread("data\\real_images\\Eoin_Macken.jpg")


convert_tensor = transforms.ToTensor()

image = convert_tensor(image)
image = image.cuda()


C:\Users\Ahmed\AppData\Local\Temp\ipykernel_15752\92410278.py:6: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = image.resize((64, 64), Image.ANTIALIAS)


In [20]:
f_net_res = f_net.forward(image)

In [21]:
def predict(tensor, model):
    yhat = model(tensor.unsqueeze(0))
    yhat = yhat.clone().detach()
    return yhat

In [22]:
predict(f_net_res, d_net).item() # predict el image fake bnsbt ad eh.

-2.4664344787597656

**We have 20 images (10 fake and 10 real)**

**calculate the identification accuracy for all the images.**

In [30]:
import glob
fake_images = glob.glob("data\\fake_images\\*")
real_images = glob.glob("data\\real_images\\*")
paper_images = glob.glob("data\\paper_images\\*")

our_model_list = []
paper_model_list = []
for i in range(10):
    our_model_list.append([fake_images[i],'fake'])
    our_model_list.append([real_images[i],'real'])
    paper_model_list.append([paper_images[i],'fake'])
    paper_model_list.append([real_images[i],'real'])

In [35]:
for i in our_model_list:
    print(i)

['data\\fake_images\\Alan_Tudyk.png', 'fake']
['data\\real_images\\Alan_Tudyk.jpg', 'real']
['data\\fake_images\\Alicja_Bachleda.png', 'fake']
['data\\real_images\\Alicja_Bachleda.jpg', 'real']
['data\\fake_images\\Bingbing_Li.png', 'fake']
['data\\real_images\\Damian_Lewis.jpg', 'real']
['data\\fake_images\\Damian_Lewis.png', 'fake']
['data\\real_images\\Eoin_Macken.jpg', 'real']
['data\\fake_images\\Eoin_Macken.png', 'fake']
['data\\real_images\\Gilles_Marini.JPG', 'real']
['data\\fake_images\\Gilles_Marini.png', 'fake']
['data\\real_images\\Li_Bingbing.png', 'real']
['data\\fake_images\\Moon_Bloodgood.png', 'fake']
['data\\real_images\\Moon_Bloodgood.jpg', 'real']
['data\\fake_images\\Suraj_Sharma.png', 'fake']
['data\\real_images\\Suraj_Sharma.jpg', 'real']
['data\\fake_images\\Witney_Carson.png', 'fake']
['data\\real_images\\Witney_Carson.jpg', 'real']
['data\\fake_images\\Woody_Harrelson.png', 'fake']
['data\\real_images\\Woody_Harrelson.jpg', 'real']


# For Our Model

In [37]:
cnt = 0
for image in our_model_list:
    img = imread(image[0])
    img = convert_tensor(img)
    img = img.cuda()
    f_net_res = f_net.forward(img)
    res = predict(f_net_res, d_net).item()
    if res >= 0.5 and image[1] == 'fake':
        print('fake image detected')
        cnt+= 1
    elif res < 0.5 and image[1] == 'real':
        print('real image detected')
        cnt+= 1

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_15752\92410278.py:6: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = image.resize((64, 64), Image.ANTIALIAS)


fake image detected
real image detected
real image detected
real image detected
fake image detected
real image detected
real image detected
real image detected
fake image detected
real image detected
real image detected
fake image detected
real image detected
real image detected


In [39]:
identification_Accuracy_our_model = cnt/20*100
identification_Accuracy_our_model

70.0